# BHBT — Hoofdstuk 1 — Oefening 1  
## Schematisch model van het brouwproces (input–proces–output)

**Doel:** een vereenvoudigd maar kwantitatief model opbouwen dat toont hoe variatie in één inputparameter zich doorheen het brouwproces kan voortplanten naar productuitkomsten (bv. **ABV** en **einddensiteit**).

> Deze notebook is didactisch: het model is bewust eenvoudig, maar procesrelevant.

---

### Wat ga je doen?
1. Het brouwproces voorstellen als een keten van **opeenvolgende blokken**.  
2. Een eenvoudige **massabalans op extract** implementeren.  
3. Variatie introduceren in één inputparameter (bv. **extractopbrengst** of **vergistingsgraad**).  
4. Het effect visualiseren op outputparameters (**OG**, **FG**, **ABV**, opbrengst).  
5. Reflecteren over aannames en systeemgrenzen.

---

### Repository
Volledige code en updates: **https://github.com/ronniewillaert/BHBT-Python-oefeningen**


## 0. Installatie/omgeving
Deze notebook draait in **Google Colab** (geen lokale installatie vereist).

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

## 1. Procesmodel als keten van blokken
We modelleren een **vereenvoudigde** keten: 

**Mout → Extract in wort (maischen/lauter) → Verdunning/verdamping (koken) → Vergisting → Product**

Elke stap is een blok dat inputs omzet naar outputs.

In [ ]:
# Helper: eenvoudige formules (didactische benaderingen)

def extract_mass_from_malt(m_malt_kg, extract_yield_kg_per_kg=0.75):
    """Massa vergistbaar+onvergistbaar extract (kg) uit mout (kg).
    extract_yield_kg_per_kg is een globale opbrengst (moutkwaliteit + procesrendement)."""
    return m_malt_kg * extract_yield_kg_per_kg

def og_from_extract(E_kg, V_l, k_points=384):
    """Zeer eenvoudige benadering: OG-points ~ k_points * (kg extract / liter).
    k_points is een calibratieconstante (didactisch)."""
    points = k_points * (E_kg / V_l)
    OG = 1.0 + points/1000.0
    return OG

def boil_volume_change(V_pre_l, boil_off_fraction=0.08):
    """Volume na koken met fractie verdamping (bv. 8%)."""
    return V_pre_l * (1.0 - boil_off_fraction)

def fg_from_og_attenuation(OG, attenuation=0.78):
    """Vereenvoudigd: FG = 1 + (OG-1)*(1-attenuation)."""
    return 1.0 + (OG - 1.0) * (1.0 - attenuation)

def abv_from_og_fg(OG, FG, k=131.25):
    """Klassieke benadering: ABV ≈ (OG - FG) * 131.25"""
    return (OG - FG) * k

## 2. Baseline parameters
Kies een **baseline** brouwscenario. Je kan later variëren.

In [ ]:
# Baseline parameters (pas gerust aan)
params = {
    "m_malt_kg": 5.0,                 # kg mout
    "extract_yield_kg_per_kg": 0.75,  # globale extractopbrengst
    "V_preboil_l": 28.0,              # liter voor koken
    "boil_off_fraction": 0.10,        # 10% verdamping
    "attenuation": 0.78               # schijnbare vergistingsgraad (0-1)
}

params

### Baseline berekening

In [ ]:
def run_model(p):
    # 1) Extract uit mout
    E = extract_mass_from_malt(p["m_malt_kg"], p["extract_yield_kg_per_kg"])
    # 2) OG vóór koken op basis van preboil volume (didactisch)
    OG_pre = og_from_extract(E, p["V_preboil_l"])
    # 3) Volume na koken
    V_post = boil_volume_change(p["V_preboil_l"], p["boil_off_fraction"])
    # 4) OG na koken stijgt door concentratie (zelfde extractmassa, minder volume)
    OG_post = og_from_extract(E, V_post)
    # 5) Vergisting
    FG = fg_from_og_attenuation(OG_post, p["attenuation"])
    ABV = abv_from_og_fg(OG_post, FG)
    return {
        "E_kg": E,
        "V_postboil_l": V_post,
        "OG_post": OG_post,
        "FG": FG,
        "ABV_percent": ABV
    }

baseline = run_model(params)
baseline

## 3. Variatie (parameter sweep)
We variëren één inputparameter en volgen de output.

Kies hieronder welke parameter je wil sweeppen: 
- `extract_yield_kg_per_kg` (grondstof + procesrendement)
- `attenuation` (gistperformantie)
- `boil_off_fraction` (kookintensiteit)

Je kan ook zelf een andere parameter toevoegen.

In [ ]:
def sweep_parameter(base_params, key, values):
    out = []
    for v in values:
        p = dict(base_params)
        p[key] = float(v)
        r = run_model(p)
        out.append((v, r))
    return out

# Kies sweep
sweep_key = "attenuation"  # <- wijzig naar "extract_yield_kg_per_kg" of "boil_off_fraction"

# Definieer sweep range
if sweep_key == "extract_yield_kg_per_kg":
    values = np.linspace(0.65, 0.85, 41)
elif sweep_key == "attenuation":
    values = np.linspace(0.65, 0.85, 41)
elif sweep_key == "boil_off_fraction":
    values = np.linspace(0.05, 0.15, 41)
else:
    raise ValueError("Onbekende sweep_key")

results = sweep_parameter(params, sweep_key, values)

# Extract arrays for plotting
x = np.array([v for v, _ in results])
OG = np.array([r["OG_post"] for _, r in results])
FG = np.array([r["FG"] for _, r in results])
ABV = np.array([r["ABV_percent"] for _, r in results])

x[:3], ABV[:3]

### Visualisatie: output versus inputvariatie

In [ ]:
plt.figure()
plt.plot(x, ABV)
plt.xlabel(sweep_key)
plt.ylabel("ABV (%)")
plt.title("Effect van inputvariatie op ABV")
plt.grid(True)
plt.show()

plt.figure()
plt.plot(x, OG, label="OG (post-boil)")
plt.plot(x, FG, label="FG")
plt.xlabel(sweep_key)
plt.ylabel("SG")
plt.title("Effect van inputvariatie op OG/FG")
plt.grid(True)
plt.legend()
plt.show()

## 4. Gevoeligheid (lokale sensitiviteit)
Een eenvoudige maat: **helling** rond het baselinepunt.

In [ ]:
# Zoek index dichtst bij baselinewaarde
x0 = params[sweep_key]
i0 = int(np.argmin(np.abs(x - x0)))

def local_slope(x, y, i):
    # centrale differentie als mogelijk
    if 0 < i < len(x)-1:
        return (y[i+1] - y[i-1]) / (x[i+1] - x[i-1])
    elif i == 0:
        return (y[1] - y[0]) / (x[1] - x[0])
    else:
        return (y[-1] - y[-2]) / (x[-1] - x[-2])

s_abv = local_slope(x, ABV, i0)
s_fg  = local_slope(x, FG, i0)

print(f"Baseline {sweep_key} = {x0:.4f}")
print(f"Lokale gevoeligheid d(ABV)/d({sweep_key}) ≈ {s_abv:.3f}")
print(f"Lokale gevoeligheid d(FG)/d({sweep_key}) ≈ {s_fg:.4f}")

## 5. Interpretatie in procescontext

Schrijf (kort) je interpretatie:
- Welke processtap “versterkt” de variatie (bv. concentratie door koken)?
- Welke parameter is het meest “kritisch” voor productvariatie in dit model?
- Welke procesmetingen/regelsystemen zouden in realiteit variabiliteit dempen?

> Tip: koppel je antwoord aan **variabiliteit**, **schaal** en **controle** (concepten uit Hoofdstuk 1).


## 6. Modelkritiek (verplicht)

Duid aan (minstens 3) welke aannames in dit model in realiteit problematisch zijn, en waarom.

Voorbeelden:
- extractopbrengst is niet constant (afhankelijk van schroten, maischschema, pH, enzymactiviteit)
- OG–extract relatie is niet exact lineair
- vergistingsgraad is niet constant (gistgezondheid, voeding, temperatuur, zuurstof, stress)
- verdamping hangt af van ketelgeometrie, warmte-inbreng, tijd, atmosferische condities
- geen tijdsdynamica (kinetiek) en geen feedback (procescontrole)


## 7. Uitbreidingen (optioneel, voor gevorderden)

1. Voeg een blok toe voor **dilutie** (top-up water) en kijk wat dat doet met OG/ABV.  
2. Introduceer onzekerheid in *twee* parameters tegelijk (Monte Carlo) en plot een verdelingsgrafiek van ABV.  
3. Vervang het lineaire OG-model door een empirische dichtheidsrelatie (met literatuur/meetdata) en vergelijk.

---  
### Kernboodschap  
Brouwen is een gekoppeld systeem: kleine inputvariaties kunnen via processtappen doorwerken naar merkbare productvariatie. Engineering start bij het expliciet maken van die keten, inclusief aannames en onzekerheid.
